In [74]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from imblearn.ensemble import BalancedBaggingClassifier
from sklearn.metrics import make_scorer,accuracy_score, classification_report, balanced_accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.feature_selection import SelectKBest, f_classif, mutual_info_classif, RFECV
from deap import creator, base, tools, algorithms
import random
import re
import warnings
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder, PowerTransformer
from ipaddress import ip_address, ip_network, ip_interface, ip_address
from sklearn.experimental import enable_iterative_imputer 
from sklearn.impute import IterativeImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.multiclass import OneVsRestClassifier
import pickle
import cupy as cp
from sklearn.gaussian_process.kernels import RBF, WhiteKernel, RationalQuadratic
from sklearn.gaussian_process import GaussianProcessRegressor
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)

In [16]:
train = pd.read_csv('/kaggle/input/objective-quest-dataquest/train.csv')
train.drop(columns=['id'], inplace=True)
traffic = train['traffic']

In [28]:
train.columns

Index(['origin_host', 'origin_port', 'response_host', 'response_port',
       'flow_duration', 'forward_packets_per_sec', 'backward_packets_per_sec',
       'flow_packets_per_sec', 'down_up_ratio', 'flow_FIN_flags',
       'flow_SYN_flags', 'flow_RST_flags', 'forward_PSH_flags',
       'backward_PSH_flags', 'flow_ACK_flags', 'forward_URG_flags',
       'backward_URG_flags', 'flow_CWR_flags', 'flow_ECE_flags',
       'forward_pkts_payload', 'backward_pkts_payload', 'flow_pkts_payload',
       'forward_iat', 'backward_iat', 'flow_iat', 'payload_bytes_per_sec',
       'forward_subflow_packets', 'backward_subflow_packets',
       'forward_subflow_bytes', 'backward_subflow_bytes', 'forward_bulk_bytes',
       'backward_bulk_bytes', 'forward_bulk_packets', 'backward_bulk_packets',
       'forward_bulk_rate', 'backward_bulk_rate', 'active', 'idle',
       'forward_initial_window_size', 'backward_initial_window_size',
       'forward_last_window_size', 'traffic'],
      dtype='object')

In [14]:
test = pd.read_csv('/kaggle/input/objective-quest-dataquest/test.csv')
test.drop(columns=['id'], inplace=True)

In [15]:
def attack(df):

    df['host_probing'] = 0
    df.loc[(df['origin_host'] == '103.255.15.150') & (df['response_host'] == '128.199.242.104'), 'host_probing'] = 1

    df['host_crypto'] = 0
    df.loc[(df['origin_host'] == '103.255.15.42') & (df['response_host'] == '103.255.15.255'), 'host_crypto'] = 1

    df['origin_bruteforce'] = 0
    df.loc[(df['origin_host'] == '103.255.15.23'), 'origin_bruteforce'] = 1

    df['response_bruteforce'] = 0
    df.loc[(df['response_host'] == '128.199.88.81'), 'response_bruteforce'] = 1

    df['response_bruteforces'] = 0
    df.loc[df['response_host'] == '128.199.242.104', 'response_bruteforces'] = 1

    df['attack_port'] = 0
    df.loc[(df['origin_port'] == 137) | (df['origin_port'] == 138) | (df['response_port'] == 137) | (df['response_port'] == 138) | (df['response_port'] == 443), 'attack_port'] = 1

    return df

In [57]:
def calculate_zero_col_percentage(df, cols, name):
    # Hitung jumlah nilai 0 per baris menggunakan vektorisasi, lalu bagi dengan jumlah kolom untuk mendapatkan persentase
    df[f'{name}'] = (df.loc[:, cols] == 0).sum(axis=1) / len(cols) * 100
    return df

def zero_col_perc_flag(df):
    name = 'zero_col_perc_flag'
    cols = ['flow_FIN_flags', 'forward_PSH_flags', 'backward_PSH_flags', 'flow_ACK_flags', 'flow_SYN_flags', 'flow_RST_flags']
    return calculate_zero_col_percentage(df, cols, name)

def zero_col_perc_att(df):
    name = 'zero_col_perc_att'
    cols = ['host_probing','host_crypto','origin_bruteforce','response_bruteforce','response_bruteforces','attack_port']
    return calculate_zero_col_percentage(df, cols, name)

In [17]:
def categorize_ports(df, origin_col='origin_port', response_col='response_port'):
    # Definisikan kategori port dalam dictionary untuk pemetaan
    port_category_map = {
        80: 1, 443: 1,  # HTTP/HTTPS
        20: 2, 21: 2,  # FTP
        22: 3, 23: 3,  # SSH/Telnet
        25: 4, 465: 4, 587: 4,  # SMTP
        53: 5,  # DNS
        67: 6, 68: 6,  # DHCP
        110: 7, 995: 7,  # POP3
        143: 8, 993: 8,  # IMAP
        137: 9, 138: 9, 139: 9,  # NetBIOS
    }

    # Kategorisasi port asal dan tujuan
    df['origin_port_cat'] = df[origin_col].fillna(0).astype(int).apply(
        lambda port: port_category_map.get(port, 10 if port < 1024 else 11 if port <= 49151 else 12))
    
    df['response_port_cat'] = df[response_col].fillna(0).astype(int).apply(
        lambda port: port_category_map.get(port, 10 if port < 1024 else 11 if port <= 49151 else 12))

    # Optimisasi kombinasi kategori port dengan menggunakan vektorisasi
    cond_same_port = df['origin_port_cat'] == df['response_port_cat']
    cond_common_service = df['origin_port_cat'].isin([1, 2, 3, 4, 5, 6, 7, 8]) & df['response_port_cat'].isin([1, 2, 3, 4, 5, 6, 7, 8])
    cond_unknown_port = (df['origin_port_cat'] == 0) | (df['response_port_cat'] == 0)
    df['port_cat'] = 0
    df.loc[cond_same_port, 'port_cat'] = 13
    df.loc[cond_common_service, 'port_cat'] = 14
    df.loc[cond_unknown_port, 'port_cat'] = 15
    df['port_cat'].fillna(16, inplace=True)  # Sisanya diberi kategori 16

    return df

In [18]:
def get_network(ip):
    try:
        return ip_interface(ip + '/24').network
    except ValueError:
        return None

# Fungsi untuk memeriksa apakah dua IP berada dalam subnet yang sama
def check_same_network(df):
    # Hitung jaringan untuk semua IP
    df['origin_network'] = df['origin_host'].apply(get_network)
    df['response_network'] = df['response_host'].apply(get_network)

    df['same_network'] = (df['origin_network'] == df['response_network']).astype(int)
    df.drop(columns=['origin_network', 'response_network'], inplace=True)
    return df

In [19]:
# Regex untuk memvalidasi format IP
ip_pattern = re.compile(r"^\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}$")

def classify_ip(ip): 
    if not ip_pattern.match(str(ip)):
        # Jika format IP tidak valid, tetapkan nilai default
        return 0, 0, 0, 0
    
    try:
        ip_obj = ip_address(ip)
        
        # Konversi IP ke integer 32-bit
        ip_int = int(ip_obj)
        
        # Tentukan kelas jaringan
        if ip_obj.is_private:
            network_class = 1  # Private
        elif ip_obj.is_global:
            if ip_int >> 24 < 224:
                network_class = 2  # Public (Unicast)
            elif ip_int >> 24 < 240:
                network_class = 3  # Multicast
            else:
                network_class = 4  # Reserved
        else:
            network_class = 0  # Unknown/Special
        
        # Hitung prefix length berdasarkan kelas
        if network_class == 2:
            if ip_int >> 24 < 128:
                prefix_length = 8
            elif ip_int >> 24 < 192:
                prefix_length = 16
            else:
                prefix_length = 24
        else:
            prefix_length = 0  # Tidak relevan untuk kelas lain
        
        # Hitung main network, subnet, dan host
        if prefix_length > 0:
            network = ip_network(f"{ip}/{prefix_length}", strict=False)
            main_network = int(network.network_address)
            subnet = (ip_int & int(network.hostmask)) >> (32 - prefix_length - 8)
            host = ip_int & 255  # Selalu ambil oktet terakhir sebagai host
        else:
            main_network = ip_int
            subnet = 0
            host = 0
        
        return network_class, main_network, subnet, host
    
    except ValueError:
        # Jika ada error dalam pemrosesan IP, tetapkan nilai default
        return 0, 0, 0, 0

# Fungsi untuk memecah kolom IP menjadi beberapa kolom baru
def split_ip_address(data, ip_column):
    data['network_class'], data['main_network'], data['subnet'], data['host'] = zip(*data[ip_column].apply(classify_ip))
    return data

def apply_split_ip_address(data):
    data = split_ip_address(data, 'origin_host')
    data = split_ip_address(data, 'response_host')
    return data

In [73]:
train.loc[train['down_up_ratio'] == 0]

,origin_host,origin_port,response_host,response_port,flow_duration,forward_packets_per_sec,backward_packets_per_sec,flow_packets_per_sec,down_up_ratio,flow_FIN_flags,...,forward_bulk_packets,backward_bulk_packets,forward_bulk_rate,backward_bulk_rate,active,idle,forward_initial_window_size,backward_initial_window_size,forward_last_window_size,traffic
12,142.252.253.26,27849,103.255.15.23,3074,0.000000,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,NaN,0.0,0.0,NaN,65535.0,0.0,65535.0,Background
13,103.255.15.20,57931,103.255.15.255,1947,0.000000,0.000000,0.0,NaN,0.0,0.0,...,0.0,0.0,NaN,NaN,0.0,NaN,0.0,0.0,0.0,Background
18,142.252.253.26,31750,103.255.15.23,80,0.000000,0.000000,0.0,NaN,0.0,0.0,...,0.0,0.0,NaN,0.0,0.0,NaN,29200.0,0.0,29200.0,Background
25,103.255.15.20,56974,103.255.15.255,1947,0.000000,0.000000,0.0,0.000000,0.0,NaN,...,0.0,0.0,NaN,0.0,NaN,NaN,0.0,0.0,0.0,Benign
39,103.255.15.65,5678,255.255.255.255,5678,0.000000,0.000000,0.0,NaN,0.0,0.0,...,0.0,0.0,NaN,NaN,NaN,0.000000e+00,0.0,0.0,0.0,Background
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
416439,103.255.15.20,56438,103.255.15.255,1947,0.000000,0.000000,0.0,0.000000,0.0,0.0,...,NaN,0.0,NaN,0.0,0.0,NaN,29200.0,0.0,29200.0,Background
416452,103.255.15.20,50602,255.255.255.255,1947,NaN,0.000000,NaN,NaN,0.0,NaN,...,0.0,0.0,0.0,NaN,0.0,0.000000e+00,0.0,0.0,0.0,Background
416454,103.210.35.245,5678,255.255.255.255,5678,0.000000,0.000000,0.0,0.000000,0.0,NaN,...,NaN,0.0,NaN,0.0,0.0,0.000000e+00,0.0,0.0,0.0,Benign
416457,103.255.15.150,49506,8.8.8.8,53,0.000015,133152.507937,0.0,133152.507937,0.0,0.0,...,NaN,0.0,0.0,0.0,NaN,0.000000e+00,0.0,0.0,0.0,Benign


Background             18630
Benign                 14781
XMRIGCC CryptoMiner       49
Bruteforce                 2
Name: traffic, dtype: int64

In [88]:
# Fungsi untuk mengimputasi nilai pada kolom-kolom yang terkait dengan packets per second
def impute_speed_values(df):
    # Misalkan nama kolom
    col_backward = 'backward_packets_per_sec'
    col_forward = 'forward_packets_per_sec'
    col_ratio = 'down_up_ratio'
    col_flow = 'flow_packets_per_sec'

    df.loc[
    (df[col_backward] == 0) & (df[col_forward] == 0), 
         [col_ratio, col_flow]
    ] = 0
    
    df.loc[
    (df[col_ratio] == 0), 
         [col_forward]
    ] = 0
    
    df.loc[
    (df[col_forward] == 0), 
         [col_ratio]
    ] = 0
    

    df.loc[df[col_backward].isna() & df[col_forward].notna() & df[col_flow].notna(), col_backward] = df[col_flow] - df[col_forward]
    df.loc[df[col_backward].isna() & df[col_forward].notna() & df[col_ratio].notna(), col_backward] = df[col_ratio] * df[col_forward]
    df.loc[df[col_forward].isna() & df[col_backward].notna() & df[col_flow].notna(), col_forward] = df[col_flow] - df[col_backward]
    if (df[col_ratio]>0).any():
      df.loc[df[col_forward].isna() & df[col_backward].notna() & df[col_ratio].notna(), col_forward] = df[col_backward] / df[col_ratio]
    df.loc[df[col_flow].isna() & df[col_backward].notna() & df[col_forward].notna(), col_flow] = df[col_backward] + df[col_forward]
    if (df[col_forward]>0).any():
     df.loc[df[col_flow].isna() & df[col_backward].notna() & df[col_forward].notna(), col_ratio] = df[col_backward] / df[col_forward]
    
   # Impute if 2 column missing
    mask = df[col_backward].isna() & df[col_forward].isna() & df[col_ratio].notna() & df[col_flow].notna()
    df.loc[mask, col_forward] = df.loc[mask, col_flow] / (df.loc[mask, col_ratio] + 1)
    df.loc[mask, col_backward] = df.loc[mask, col_flow] - df.loc[mask, col_forward]
    
    mask = df[col_ratio].isna() & df[col_forward].isna() & df[col_backward].notna() & df[col_flow].notna()
    df.loc[mask, col_forward] = df.loc[mask, col_flow] - df.loc[mask, col_backward]
    if (df[col_forward]>0).any():
     df.loc[mask, col_ratio] = df.loc[mask, col_backward] / df.loc[mask, col_forward]

    mask = df[col_ratio].isna() & df[col_backward].isna() & df[col_forward].notna() & df[col_flow].notna()
    df.loc[mask, col_backward] = df.loc[mask, col_flow] - df.loc[mask, col_forward]
    if (df[col_forward]>0).any():
     df.loc[mask, col_ratio] = df.loc[mask, col_backward] / df.loc[mask, col_forward]
    
    mask = df[col_forward].isna() & df[col_flow].isna() & df[col_ratio].notna() & df[col_backward].notna()
    if (df[col_ratio]>0).any():
         df.loc[mask, col_forward] = df.loc[mask, col_backward] / (df.loc[mask, col_ratio])
    df.loc[mask, col_flow] = df.loc[mask, col_backward] + (df.loc[mask, col_forward])
    
    mask = df[col_backward].isna() & df[col_flow].isna() & df[col_ratio].notna() & df[col_forward].notna()
    df.loc[mask, col_backward] = df.loc[mask, col_ratio] * (df.loc[mask, col_forward])
    df.loc[mask, col_flow] = df.loc[mask, col_backward] + (df.loc[mask, col_forward])
    
    mask = df[col_ratio].isna() & df[col_flow].isna() & df[col_backward].notna() & df[col_forward].notna()
    df.loc[mask, col_flow] = df.loc[mask, col_backward] + (df.loc[mask, col_forward])
    if (df[col_forward]>0).any():
     df.loc[mask, col_ratio] = df.loc[mask, col_backward] / df.loc[mask, col_forward]
    
    #if inf downUpratio change to 0
    df.loc[df[col_ratio] == float('inf'), col_ratio] = 0
    return df

In [62]:
def window_impute(df):
    bwd_init_window = df['backward_initial_window_size']
    fwd_init_window = df['forward_initial_window_size']
    fwd_last_window = df['forward_last_window_size']
    response_port = df['response_port']

    # Periksa apakah semua nilai adalah NaN
    if pd.isna(bwd_init_window) and pd.isna(fwd_init_window) and pd.isna(fwd_last_window):
        return 0, 0, 0

    # Imputasi untuk backward_initial_window_size == 0
    if bwd_init_window == 0:
        if not pd.isna(fwd_init_window) and pd.isna(fwd_last_window):
            fwd_last_window = fwd_init_window
        elif pd.isna(fwd_init_window) and not pd.isna(fwd_last_window):
            fwd_init_window = fwd_last_window
        else:
            if response_port == 53:
                fwd_init_window = 0
                fwd_last_window = 0
            else:
                fwd_init_window = 29200
                fwd_last_window = 29200

    # Imputasi untuk backward_initial_window_size == NaN
    elif pd.isna(bwd_init_window):
        if fwd_init_window == 0:
            if pd.isna(fwd_last_window):
                fwd_last_window = 0
                bwd_init_window = 0
            else:
                bwd_init_window = 0

    # Imputasi untuk forward_initial_window_size == NaN
    elif pd.isna(fwd_init_window):
        if fwd_last_window in [5840, 1024, 8192, 65535, 60, 28800]:
            fwd_init_window = fwd_last_window
            bwd_init_window = 0
        else:
            fwd_init_window = 29200
            bwd_init_window = 65160

    return bwd_init_window, fwd_init_window, fwd_last_window

# Fungsi untuk menerapkan window_impute ke seluruh DataFrame
def apply_window_impute(df):
    result = df.apply(window_impute, axis=1)
    df['backward_initial_window_size'], df['forward_initial_window_size'], df['forward_last_window_size'] = zip(*result)
    return df

In [21]:
def down_up_ratio_prob(df):
  df['down_up_ratio_prob'] = 0
  df.loc[(df['down_up_ratio'] <= 2) & (df['down_up_ratio'] > 1), 'down_up_ratio_prob'] = 1
  return df

In [33]:
def ip_to_int(df):
    df.loc[:, 'origin_host'] = df['origin_host'].map(lambda x: int(ip_address(x)))
    df.loc[:, 'response_host'] = df['response_host'].map(lambda x: int(ip_address(x)))
    return df

In [23]:
def remove_nunique_1(df):
    for col in df.columns:
        if df[col].nunique() == 1:
            df.drop(col, axis=1, inplace=True)
    return df

In [89]:
def preprocess(df):
    copy = df.copy(deep = True)
    function = [attack, 
                zero_col_perc_att,
                zero_col_perc_flag, 
                categorize_ports,
                check_same_network,
                down_up_ratio_prob,
                apply_split_ip_address,
                ip_to_int, 
                impute_speed_values,
                apply_window_impute]
    for func in function:
        copy = func(copy)
    return copy

col_exc_traffic = [col for col in train.columns if 'traffic' not in col]
print(col_exc_traffic)
train_pre = preprocess(train[col_exc_traffic])
test_pre = preprocess(test)

['origin_host', 'origin_port', 'response_host', 'response_port', 'flow_duration', 'forward_packets_per_sec', 'backward_packets_per_sec', 'flow_packets_per_sec', 'down_up_ratio', 'flow_FIN_flags', 'flow_SYN_flags', 'flow_RST_flags', 'forward_PSH_flags', 'backward_PSH_flags', 'flow_ACK_flags', 'forward_URG_flags', 'backward_URG_flags', 'flow_CWR_flags', 'flow_ECE_flags', 'forward_pkts_payload', 'backward_pkts_payload', 'flow_pkts_payload', 'forward_iat', 'backward_iat', 'flow_iat', 'payload_bytes_per_sec', 'forward_subflow_packets', 'backward_subflow_packets', 'forward_subflow_bytes', 'backward_subflow_bytes', 'forward_bulk_bytes', 'backward_bulk_bytes', 'forward_bulk_packets', 'backward_bulk_packets', 'forward_bulk_rate', 'backward_bulk_rate', 'active', 'idle', 'forward_initial_window_size', 'backward_initial_window_size', 'forward_last_window_size']


In [9]:
train_pre.to_csv('../data/train_pre.csv', index=False)
test_pre.to_csv('../data/test_pre.csv', index=False)

In [62]:
train_pre = pd.read_csv('/kaggle/input/train-pre-and-test-pre-traffic-analysis/train_pre.csv')
test_pre = pd.read_csv('/kaggle/input/train-pre-and-test-pre-traffic-analysis/test_pre.csv')

In [63]:
print(len(train_pre.columns)==len(test_pre.columns))

True


In [8]:
train_pre.drop(columns=['host_background', 'normal_port'], inplace=True)
test_pre.drop(columns=['host_background', 'normal_port'], inplace=True)

In [64]:
train_pre.head()

,origin_host,origin_port,response_host,response_port,flow_duration,forward_packets_per_sec,backward_packets_per_sec,flow_packets_per_sec,down_up_ratio,flow_FIN_flags,flow_SYN_flags,flow_RST_flags,forward_PSH_flags,backward_PSH_flags,flow_ACK_flags,forward_URG_flags,backward_URG_flags,flow_CWR_flags,flow_ECE_flags,forward_pkts_payload,backward_pkts_payload,flow_pkts_payload,forward_iat,backward_iat,flow_iat,payload_bytes_per_sec,forward_subflow_packets,backward_subflow_packets,forward_subflow_bytes,backward_subflow_bytes,forward_bulk_bytes,backward_bulk_bytes,forward_bulk_packets,backward_bulk_packets,forward_bulk_rate,backward_bulk_rate,active,idle,forward_initial_window_size,backward_initial_window_size,forward_last_window_size,host_probing,host_crypto,origin_bruteforce,response_bruteforce,response_bruteforces,attack_port,zero_col_perc_att,zero_col_perc_flag,origin_port_cat,response_port_cat,port_cat,same_network,down_up_ratio_prob,network_class,main_network,subnet,host
0,1744768795,45314,1744768791,42001,0.000090,11125.474801,11125.474801,22250.949602,1.00,0.0,1.0,NaN,NaN,NaN,1.0,0.0,0.0,NaN,0.0,0.0,0.000000,0.000000,0.000000e+00,NaN,89.883804,0.000000,1.000000,NaN,0.000000,0.000000,0.0,0.0,0.0,0.0,NaN,0.0,89.883804,NaN,29200.0,NaN,NaN,0,0,0,0,0,0,100.000000,16.666667,11,11,13,1,0,2,1728053248,255,23
1,1744768795,45060,1744768791,42000,0.000033,30174.848921,30174.848921,60349.697842,1.00,NaN,1.0,NaN,0.0,0.0,NaN,0.0,NaN,0.0,0.0,0.0,0.000000,0.000000,0.000000e+00,0.000000e+00,33.140182,0.000000,NaN,1.0,0.000000,NaN,0.0,0.0,0.0,0.0,0.0,0.0,33.140182,NaN,29200.0,0.0,29200.0,0,0,0,0,0,0,100.000000,33.333333,11,11,13,1,0,2,1728053248,255,23
2,1744768791,3440,875368640,443,61.977190,0.322699,0.242025,0.564724,0.75,2.0,2.0,0.0,NaN,6.0,NaN,0.0,0.0,0.0,0.0,70.8,408.733333,215.628571,3.261959e+06,4.408952e+06,NaN,121.770606,2.857143,NaN,202.285714,875.857143,0.0,0.0,NaN,0.0,NaN,0.0,401089.021138,9.861604e+06,29200.0,NaN,90.0,0,0,1,0,0,1,66.666667,16.666667,11,1,0,0,0,2,872415232,45,192
3,1744768918,56132,134744072,53,0.024249,82.478177,82.478178,164.956355,1.00,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,36.0,NaN,54.750000,1.192093e+01,NaN,8082.946142,9031.360437,NaN,2.0,72.000000,147.000000,0.0,NaN,NaN,0.0,0.0,0.0,24248.838425,NaN,0.0,0.0,0.0,0,0,0,0,0,0,100.000000,83.333333,12,5,0,0,0,2,134217728,8,8
4,1744768918,43358,134744072,53,0.027580,NaN,NaN,72.516256,NaN,NaN,0.0,0.0,0.0,0.0,NaN,0.0,0.0,NaN,0.0,36.0,NaN,65.500000,0.000000e+00,0.000000e+00,27580.022812,4749.814781,1.000000,1.0,36.000000,95.000000,0.0,0.0,0.0,0.0,NaN,0.0,27580.022812,0.000000e+00,0.0,0.0,0.0,0,0,0,0,0,0,100.000000,66.666667,11,5,0,0,0,2,134217728,8,8


In [65]:
train_pre.columns

Index(['origin_host', 'origin_port', 'response_host', 'response_port',
       'flow_duration', 'forward_packets_per_sec', 'backward_packets_per_sec',
       'flow_packets_per_sec', 'down_up_ratio', 'flow_FIN_flags',
       'flow_SYN_flags', 'flow_RST_flags', 'forward_PSH_flags',
       'backward_PSH_flags', 'flow_ACK_flags', 'forward_URG_flags',
       'backward_URG_flags', 'flow_CWR_flags', 'flow_ECE_flags',
       'forward_pkts_payload', 'backward_pkts_payload', 'flow_pkts_payload',
       'forward_iat', 'backward_iat', 'flow_iat', 'payload_bytes_per_sec',
       'forward_subflow_packets', 'backward_subflow_packets',
       'forward_subflow_bytes', 'backward_subflow_bytes', 'forward_bulk_bytes',
       'backward_bulk_bytes', 'forward_bulk_packets', 'backward_bulk_packets',
       'forward_bulk_rate', 'backward_bulk_rate', 'active', 'idle',
       'forward_initial_window_size', 'backward_initial_window_size',
       'forward_last_window_size', 'host_probing', 'host_crypto',
       'or

In [76]:
max = np.max(train_pre, axis=0)
max

origin_host                             998141876
origin_port                                 65535
response_host                           997171445
response_port                               65522
flow_duration                          17393.0306
forward_packets_per_sec                 167772.16
backward_packets_per_sec                167772.16
flow_packets_per_sec                182361.043478
down_up_ratio                                 6.0
flow_FIN_flags                               15.0
flow_SYN_flags                               23.0
flow_RST_flags                              101.0
forward_PSH_flags                          3953.0
backward_PSH_flags                        48653.0
flow_ACK_flags                           118234.0
forward_URG_flags                             2.0
backward_URG_flags                            1.0
flow_CWR_flags                                4.0
flow_ECE_flags                                2.0
forward_pkts_payload                  3718.933934


In [77]:
min = np.min(train_pre, axis=0)
min

origin_host                             0
origin_port                            21
response_host                   100473501
response_port                           0
flow_duration                         0.0
forward_packets_per_sec               0.0
backward_packets_per_sec              0.0
flow_packets_per_sec                  0.0
down_up_ratio                         0.0
flow_FIN_flags                        0.0
flow_SYN_flags                        0.0
flow_RST_flags                        0.0
forward_PSH_flags                     0.0
backward_PSH_flags                    0.0
flow_ACK_flags                        0.0
forward_URG_flags                     0.0
backward_URG_flags                    0.0
flow_CWR_flags                        0.0
flow_ECE_flags                        0.0
forward_pkts_payload                  0.0
backward_pkts_payload                 0.0
flow_pkts_payload                     0.0
forward_iat                           0.0
backward_iat                      

In [68]:
def check_inf_col(df):
    for col in df.select_dtypes(include=['int64', 'float64']).columns:
        if np.isinf(df[col]).any():
            print(col)

check_inf_col(train_pre)
check_inf_col(test_pre)

In [69]:
def check_nunique_1(df):
    for col in df.columns:
        if df[col].nunique() == 1:
            print(col)
check_nunique_1(train_pre)
check_nunique_1(test_pre)

In [104]:
def check_nunique_col(df):
    for col in df.columns:
        print(f'col: {df[col].nunique()}')
check_nunique_col(train_pre)
check_nunique_col(test_pre)

col: 2389

col: 61264

col: 7100

col: 3540

col: 120442

col: 174824

col: 174193

col: 176807

col: 5448

col: 12

col: 15

col: 38

col: 490

col: 910

col: 1918

col: 3

col: 2

col: 5

col: 3

col: 19801

col: 27400

col: 37029

col: 122253

col: 110385

col: 146216

col: 182885

col: 2746

col: 3092

col: 11928

col: 22004

col: 783

col: 10018

col: 133

col: 2688

col: 1038

col: 20281

col: 138676

col: 27272

col: 191

col: 143

col: 2506

col: 2

col: 2

col: 2

col: 2

col: 2

col: 2

col: 4

col: 7

col: 10

col: 12

col: 3

col: 2

col: 2

col: 1

col: 2

col: 4

col: 774

col: 256

col: 256

col: 1136

col: 47651

col: 4258

col: 1369

col: 42477

col: 68298

col: 68287

col: 69275

col: 2664

col: 11

col: 16

col: 19

col: 333

col: 595

col: 1209

col: 4

col: 2

col: 5

col: 2

col: 9534

col: 12873

col: 13172

col: 41366

col: 42851

col: 54153

col: 70846

col: 1568

col: 1630

col: 5425

col: 11021

col: 246

col: 3518

col: 65

col: 1083

col: 366

col: 7970

co

In [15]:

# def iterative_imputer(train_df, test_df):
#     # Membuat instance dari kelas IterativeImputer
#     imputer = IterativeImputer(
#         estimator=xgb.XGBRegressor(tree_method='gpu_hist', random_state=42, n_estimators=100, max_depth=5, gpu_id=0, learning_rate=0.05, boostrap='gbtree'),
#         add_indicator=True,  
#         max_iter=15, 
#         random_state=42, 
#         n_nearest_features=5, 
#         tol=0.001, 
#         skip_complete=True,
#         min_value = min,
#         max_value = max,
#         initial_strategy='most_frequent',
#         missing_values = np.nan, 
#         imputation_order='ascending'
#     )
    
#     imputed_train_array = imputer.fit_transform(train_df)
#     imputed_test_array = imputer.transform(test_df)
#     original_columns = train_df.columns
#     indicator_columns = [f'missing_indicator_{i}' for i in range(imputed_train_array.shape[1] - len(original_columns))]
#     all_columns = list(original_columns) + indicator_columns
    
#     # Kembalikan hasil imputasi sebagai DataFrame dengan kolom yang benar
#     imputed_train_df = pd.DataFrame(imputed_train_array, columns=all_columns)
#     imputed_test_df = pd.DataFrame(imputed_test_array, columns=all_columns)
    
#     return imputed_train_df, imputed_test_df
# train_imputed, test_imputed = iterative_imputer(train_pre, test_pre)

In [1]:
def optimized_iterative_imputer(train_df, test_df):
    # Define the kernel for GaussianProcessRegressor
    kernel = RBF() + WhiteKernel() + RationalQuadratic()
    
    # Create an instance of GaussianProcessRegressor
    gpr = GaussianProcessRegressor(
        kernel=kernel,
        alpha=1e-10,
        normalize_y=True,
        n_restarts_optimizer=10,
        random_state=42
    )
    
    # Create an instance of IterativeImputer
    imputer = IterativeImputer(
        estimator=gpr,
        add_indicator=True,
        max_iter=20,
        random_state=42,
        n_nearest_features=10,
        tol=0.0001,
        skip_complete=True,
        min_value=min,
        max_value=max,
        initial_strategy='most_frequent',
        missing_values=np.nan,
        imputation_order='ascending'
    )
    
    # Fit and transform the training data
    imputed_train_array = imputer.fit_transform(train_df)
    
    # Transform the test data
    imputed_test_array = imputer.transform(test_df)
    
    # Prepare column names
    original_columns = train_df.columns
    indicator_columns = [f'missing_indicator_{i}' for i in range(imputed_train_array.shape[1] - len(original_columns))]
    all_columns = list(original_columns) + indicator_columns
    
    # Convert imputed arrays to DataFrames
    imputed_train_df = pd.DataFrame(imputed_train_array, columns=all_columns, index=train_df.index)
    imputed_test_df = pd.DataFrame(imputed_test_array, columns=all_columns, index=test_df.index)
    
    return imputed_train_df, imputed_test_df
train_imputed_gpr, train_imputed_gpr = optimized_iterative_imputer(train_pre, test_pre)

NameError: name 'train_pre' is not defined

In [ ]:
train_imputed_gpr.to_csv('train_imputed_gpr.csv', index=False)
test_imputed_gpr.to_csv('test_imputed_gpr.csv', index=False)

In [9]:
train_imputed_gpr = pd.read_csv('/kaggle/working/train_imputed_gpr.csv')
train_imputed_gpr = pd.read_csv('/kaggle/working/train_imputed_gpr.csv')

In [ ]:
from sklearn.preprocessing import PowerTransformer

excluded_columns = ['origin_host', 'response_host', 'forward_last_window_size', 
                    'host_probing', 'host_crypto', 'origin_bruteforce', 'network_class',
                    'response_bruteforce', 'response_bruteforces', 'attack_port', 'origin_port_cat', 'response_port_cat', 'port_cat', 'same_network', 'origin_port','host', 'response_port']
train_tf = train_imputed_gpr.copy(deep=True)
test_tf = test_imputed_gpr.copy(deep=True)
power_transformer = PowerTransformer(method='yeo-johnson', standardize=True)
for df in [train_tf, test_tf]:
    for col in df.columns:
        if col not in excluded_columns:
            if df is train_tf:
                df[col] = power_transformer.fit_transform(df[[col]])
            else:
                df[col] = power_transformer.transform(df[[col]])

In [ ]:
train_tf.to_csv('train_tf.csv', index=False)
test_tf.to_csv('test_tf.csv', index=False)

In [19]:
train_imputed

,origin_host,origin_port,response_host,response_port,flow_duration,forward_packets_per_sec,backward_packets_per_sec,flow_packets_per_sec,down_up_ratio,flow_FIN_flags,flow_SYN_flags,flow_RST_flags,forward_PSH_flags,backward_PSH_flags,flow_ACK_flags,forward_URG_flags,backward_URG_flags,flow_CWR_flags,flow_ECE_flags,forward_pkts_payload,backward_pkts_payload,flow_pkts_payload,forward_iat,backward_iat,flow_iat,payload_bytes_per_sec,forward_subflow_packets,backward_subflow_packets,forward_subflow_bytes,backward_subflow_bytes,forward_bulk_bytes,backward_bulk_bytes,forward_bulk_packets,backward_bulk_packets,forward_bulk_rate,backward_bulk_rate,active,idle,forward_initial_window_size,backward_initial_window_size,forward_last_window_size,host_probing,host_crypto,origin_bruteforce,response_bruteforce,response_bruteforces,attack_port,zero_col_perc_att,zero_col_perc_flag,origin_port_cat,response_port_cat,port_cat,same_network,down_up_ratio_prob,network_class,main_network,subnet,host,missing_indicator_0,missing_indicator_1,missing_indicator_2,missing_indicator_3,missing_indicator_4,missing_indicator_5,missing_indicator_6,missing_indicator_7,missing_indicator_8,missing_indicator_9,missing_indicator_10,missing_indicator_11,missing_indicator_12,missing_indicator_13,missing_indicator_14,missing_indicator_15,missing_indicator_16,missing_indicator_17,missing_indicator_18,missing_indicator_19,missing_indicator_20,missing_indicator_21,missing_indicator_22,missing_indicator_23,missing_indicator_24,missing_indicator_25,missing_indicator_26,missing_indicator_27,missing_indicator_28,missing_indicator_29,missing_indicator_30,missing_indicator_31,missing_indicator_32,missing_indicator_33,missing_indicator_34,missing_indicator_35,missing_indicator_36
0,1.744769e+09,45314.0,1.744769e+09,42001.0,0.000090,11125.474801,11125.474801,22250.949602,1.000000,0.000000,1.000000,0.940561,0.165281,0.898466,1.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,4.407406e+03,8.988380e+01,0.000000,1.000000,1.290115,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,18.551392,0.000000e+00,8.988380e+01,3.596850e+04,29200.0,150.511719,29096.503906,0.0,0.0,0.0,0.0,0.0,0.0,100.000000,16.666667,11.0,11.0,13.0,1.0,0.0,2.0,1.728053e+09,255.0,23.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0
1,1.744769e+09,45060.0,1.744769e+09,42000.0,0.000033,30174.848921,30174.848921,60349.697842,1.000000,0.003605,1.000000,0.940561,0.000000,0.000000,1.191376,0.000000,3.162231e-07,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,3.314018e+01,0.000000,1.251842,1.000000,0.000000,501.177734,0.00000,0.000000,0.000000,0.0,0.000000,0.000000e+00,3.314018e+01,1.849198e+05,29200.0,0.000000,29200.000000,0.0,0.0,0.0,0.0,0.0,0.0,100.000000,33.333333,11.0,11.0,13.0,1.0,0.0,2.0,1.728053e+09,255.0,23.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,1.744769e+09,3440.0,8.753686e+08,443.0,61.977190,0.322699,0.242025,0.564724,0.750000,2.000000,2.000000,0.000000,3.810586,6.000000,20.290390,0.000000,0.000000e+00,0.000000,0.000000e+00,70.800000,408.733333,215.628571,3.261959e+06,4.408952e+06,1.851626e+06,121.770606,2.857143,7.393608,202.285714,875.857143,0.00000,0.000000,0.000161,0.0,20.024353,0.000000e+00,4.010890e+05,9.861604e+06,29200.0,45790.992188,90.000000,0.0,0.0,1.0,0.0,0.0,1.0,66.666667,16.666667,11.0,1.0,0.0,0.0,0.0,2.0,8.724152e+08,45.0,192.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1.744769e+09,56132.0,1.347441e+08,53.0,0.024249,82.478177,82.478178,164.956355,1.000000,0.000000,0.006180,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,5.815200e-07,36.000000,76.655548,54.750000,1.192093e+01,5.277094e+03,8.082946e+03,90

In [21]:
check_nunique_1(train_imputed)
check_nunique_1(test_imputed)

In [18]:
X = train_imputed
y = traffic

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [34]:
# def advanced_feature_selection(X, y):
#     y == LabelEncoder().fit_transform(y)
#     selector = RFECV(
#         estimator=xgb.XGBClassifier(tree_method='gpu_hist', random_state=42, n_estimators=100, max_depth=5, gpu_id=0, learning_rate=0.05, boostrap='gbtree'),
#         step=1, cv=5, scoring='accuracy', 
#         min_features_to_select=5
#     )
#     X_new = selector.fit_transform(X, y)
#     selected_features = X.columns[selector.support_].tolist()
#     print(f"Selected {len(selected_features)} features: {selected_features}")
#     return X_new
# advanced_feature_selection(train_imputed, traffic)

Selected 83 features: ['origin_host', 'origin_port', 'response_host', 'response_port', 'flow_duration', 'forward_packets_per_sec', 'backward_packets_per_sec', 'flow_packets_per_sec', 'down_up_ratio', 'flow_SYN_flags', 'flow_RST_flags', 'forward_PSH_flags', 'backward_PSH_flags', 'flow_ACK_flags', 'forward_URG_flags', 'backward_URG_flags', 'flow_ECE_flags', 'forward_pkts_payload', 'backward_pkts_payload', 'flow_pkts_payload', 'forward_iat', 'backward_iat', 'flow_iat', 'payload_bytes_per_sec', 'forward_subflow_packets', 'backward_subflow_packets', 'forward_subflow_bytes', 'backward_subflow_bytes', 'forward_bulk_bytes', 'backward_bulk_bytes', 'forward_bulk_packets', 'backward_bulk_packets', 'forward_bulk_rate', 'active', 'idle', 'forward_initial_window_size', 'backward_initial_window_size', 'forward_last_window_size', 'host_probing', 'host_crypto', 'origin_bruteforce', 'response_bruteforce', 'response_bruteforces', 'zero_col_perc_att', 'zero_col_perc_flag', 'origin_port_cat', 'response_por

TypeError: eaSimple() got an unexpected keyword argument 'args'

In [ ]:
from sklearn.feature_selection import RFE
def optimized_feature_selection(X, y, n_features_to_select=20):
    rfc = RandomForestClassifier(n_estimators=100, n_jobs=-1, max_depth=10, random_state=42)
    rfe = RFE(estimator=rfc, n_features_to_select=n_features_to_select, step=1)
    rfe.fit(X, y)
    selected_features = X.columns[rfe.support_]
    
    return selected_features

In [ ]:
def optimized_gaussian_nb(X, y):
    # Define the parameter space
    param_dist = {
        'var_smoothing': uniform(1e-12, 1e-6),
        'priors': [None, 'balanced']
    }
    
    # Create a base GaussianNB model
    gnb = GaussianNB()
    
    # Set up RandomizedSearchCV
    random_search = RandomizedSearchCV(
        gnb,
        param_distributions=param_dist,
        n_iter=100,
        cv=5,
        scoring='accuracy',
        n_jobs=-1,
        random_state=42
    )
    
    # Fit RandomizedSearchCV
    random_search.fit(X, y)
    
    # Get the best model
    best_gnb = random_search.best_estimator_
    
    return best_gnb


In [ ]:
best_gnb = optimized_gaussian_nb(X_train, y_train)

# Make predictions
y_pred = best_gnb.predict(X_test)

In [20]:
from lightgbm import LGBMClassifier

# Use the best model to make predictions
lgb = LGBMClassifier(n_estimators=300,
                     learning_rate=0.1,
                     max_depth=9,
                     min_samples_leaf=2,
                     gamma=0,
                     reg_alpha=0,
                     reg_lambda=0.1,
                     sample_pos_weight=1,
                     verbose=-1,
                     random_state=42)

In [21]:
lgb_bbc = BalancedBaggingClassifier(estimator         = lgb,
                                    sampling_strategy = 'auto',
                                    n_jobs            = -1,
                                    random_state      = 42)

In [24]:
class AdvancedDFAGPE_with_feature_selected:
    def __init__(self, n_classifiers=10, n_generations=50):
        self.n_classifiers = n_classifiers
        self.n_generations = n_generations
        self.base_classifiers = None
        self.best_combiner = None
        self.selected_features = None
        self.label_encoder = LabelEncoder()

        # DEAP setup
        if not hasattr(creator, 'FitnessMax'):
            creator.create("FitnessMax", base.Fitness, weights=(1.0,))
        if not hasattr(creator, 'Individual'):
            creator.create("Individual", list, fitness=creator.FitnessMax)

    def create_base_classifier(self):
        return OneVsRestClassifier(lgb_bbc)

    def evalCombiner(self, individual):
        individual_gpu = cp.array(individual)
        ensemble_predictions = cp.tensordot(individual_gpu, self.base_predictions_gpu, axes=1)
        predictions = cp.argmax(ensemble_predictions, axis=1)
        accuracy = cp.mean(predictions == self.y_gpu)
        return (float(accuracy),)

    def createCombinerGraph(self):
        toolbox = base.Toolbox()
        toolbox.register("attr_float", random.random)
        toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_float, n=self.n_classifiers)
        toolbox.register("population", tools.initRepeat, list, toolbox.individual)
        toolbox.register("evaluate", self.evalCombiner)
        toolbox.register("mate", tools.cxBlend, alpha=0.5)
        toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=0.2, indpb=0.2)
        toolbox.register("select", tools.selTournament, tournsize=3)
        return toolbox

    def fit(self, X, y):
        #feature dari rfecv sebelumnya
        self.selected_features = [
            'origin_host', 'origin_port', 'response_host', 'response_port', 'flow_duration',
            'forward_packets_per_sec', 'backward_packets_per_sec', 'flow_packets_per_sec',
            'down_up_ratio', 'flow_SYN_flags', 'flow_RST_flags', 'forward_PSH_flags',
            'backward_PSH_flags', 'flow_ACK_flags', 'forward_URG_flags', 'backward_URG_flags',
            'flow_ECE_flags', 'forward_pkts_payload', 'backward_pkts_payload', 'flow_pkts_payload',
            'forward_iat', 'backward_iat', 'flow_iat', 'payload_bytes_per_sec',
            'forward_subflow_packets', 'backward_subflow_packets', 'forward_subflow_bytes',
            'backward_subflow_bytes', 'forward_bulk_bytes', 'backward_bulk_bytes',
            'forward_bulk_packets', 'backward_bulk_packets', 'forward_bulk_rate', 'active',
            'idle', 'forward_initial_window_size', 'backward_initial_window_size',
            'forward_last_window_size', 'host_probing', 'host_crypto', 'origin_bruteforce',
            'response_bruteforce', 'response_bruteforces', 'zero_col_perc_att',
            'zero_col_perc_flag', 'origin_port_cat', 'response_port_cat', 'port_cat',
            'down_up_ratio_prob', 'main_network', 'subnet', 'host', 'missing_indicator_0',
            'missing_indicator_1', 'missing_indicator_2', 'missing_indicator_4',
            'missing_indicator_5', 'missing_indicator_6', 'missing_indicator_7',
            'missing_indicator_8', 'missing_indicator_9', 'missing_indicator_10',
            'missing_indicator_11', 'missing_indicator_14', 'missing_indicator_15',
            'missing_indicator_16', 'missing_indicator_17', 'missing_indicator_18',
            'missing_indicator_21', 'missing_indicator_22', 'missing_indicator_23',
            'missing_indicator_24', 'missing_indicator_25', 'missing_indicator_26',
            'missing_indicator_27', 'missing_indicator_29', 'missing_indicator_30',
            'missing_indicator_31', 'missing_indicator_32', 'missing_indicator_33',
            'missing_indicator_34', 'missing_indicator_35', 'missing_indicator_36'
        ]
        
        X_selected = X[self.selected_features]
        y_encoded = self.label_encoder.fit_transform(y)
        
        self.base_classifiers = [self.create_base_classifier() for _ in range(self.n_classifiers)]
        base_predictions = np.array([clf.fit(X_selected, y_encoded).predict_proba(X_selected) for clf in self.base_classifiers])
        
        # Move data to GPU
        self.X_gpu = cp.array(X_selected.values)
        self.y_gpu = cp.array(y_encoded)
        self.base_predictions_gpu = cp.array(base_predictions)
        
        toolbox = self.createCombinerGraph()
        population = toolbox.population(n=50)
        
        # Run the evolutionary algorithm
        self.best_combiner = algorithms.eaSimple(
            population, toolbox, cxpb=0.5, mutpb=0.2, ngen=self.n_generations,
            stats=None, halloffame=None, verbose=False
        )[0][0]

    def predict(self, X):
        if self.base_classifiers is None or self.best_combiner is None:
            raise ValueError("Model has not been fitted yet.")
        
        X_selected = X[self.selected_features]
        base_predictions = np.array([clf.predict_proba(X_selected) for clf in self.base_classifiers])
        
        # Move predictions to GPU
        base_predictions_gpu = cp.array(base_predictions)
        best_combiner_gpu = cp.array(self.best_combiner)
        
        ensemble_predictions = cp.tensordot(best_combiner_gpu, base_predictions_gpu, axes=1)
        encoded_predictions = cp.asnumpy(cp.argmax(ensemble_predictions, axis=1))
        return self.label_encoder.inverse_transform(encoded_predictions)

    def save_model(self, filename):
        with open(filename, 'wb') as f:
            pickle.dump(self, f)

    @classmethod
    def load_model(cls, filename):
        with open(filename, 'rb') as f:
            return pickle.load(f)

In [25]:
model_lgb_bbc = AdvancedDFAGPE_with_feature_selected()
model_lgb_bbc.fit(X_train,y_train)

KeyboardInterrupt: 

In [ ]:
y_pred = model_lgb_bbc.predict(X_test)

In [ ]:
def mean_accuracy_score(y_true, y_pred):
    return (accuracy_score(y_true, y_pred) + balanced_accuracy_score(y_true, y_pred)) / 2

mean_accuracy_scorer = make_scorer(mean_accuracy_score)

def evaluation(y_test, y_pred):
    accuracy = accuracy_score(y_test, y_pred)
    balanced_accuracy = balanced_accuracy_score(y_test,y_pred)
    mean_accuracy = mean_accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')
    print(f"Accuracy: {accuracy}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"F1 Score: {f1}")
    print(f"BAS: {balanced_accuracy}")
    print(f"Mean Accuracy: {mean_accuracy}")

In [26]:
evaluation(y_test, y_pred)

NameError: name 'evaluation' is not defined

In [43]:
model_2.save_model('advanced_dfa_gpe_model.pkl')

In [ ]:
cm = confusion_matrix(y_test, y_pred_dfa)
cm

In [50]:
id_test = pd.read_csv('/kaggle/input/objective-quest-dataquest/test.csv')['id']

In [38]:
#Memuat model
# loaded_model = AdvancedDFAGPE_with_feature_selected.load_model('/kaggle/working/advanced_dfa_gpe_model.pkl')

y_pred_test = model_2.predict(test_imputed)

In [42]:
flat = pd.DataFrame(y_pred_test)
flat.value_counts()

0                  
Benign                 98368
Background             33358
Probing                 2882
Bruteforce              1851
Bruteforce-XML          1276
XMRIGCC CryptoMiner     1070
Name: count, dtype: int64

In [47]:
def convert_np_array_to_list(array):
    return array.tolist()
y_pred_test = convert_np_array_to_list(y_pred_test)

In [52]:
sub = pd.DataFrame({'id': id_test, 'traffic': y_pred_test})
sub.to_csv('sub_dfa_gpe_2.csv', index=False)